In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('NER_EXPORT/161704902/treino_1/Documentos/564.ner.csv', 
                 delimiter=';', na_values='NaN')

In [3]:
# df = df.query("Token not in ['', ' ', '\\n']")
# df = df.query("Token not in ['', ' ']")

In [4]:
df.dropna(how='any', inplace=True)

In [5]:
df.head()

,Token,Tag
0,EMENTA,O
1,:,O
2,EXECUÇÃO,O
3,PENAL,O
4,.,O


In [8]:
# df.query("Tag.str.contains('B_')", engine='python')
df.query("Tag not in 'O'")

,Token,Tag
124,HHCC,B_Precedente
125,123.382,I_Precedente
126,e,I_Precedente
127,123.425,I_Precedente
128,",",I_Precedente
129,Relatores,I_Precedente
130,a,I_Precedente
131,Ministra,I_Precedente
132,Rosa,I_Precedente
133,Weber,I_Precedente


In [7]:
df.groupby('Tag').size().reset_index(name='Count')

,Tag,Count
0,B_Doutrinador,2
1,B_Pessoa,19
2,B_Precedente,20
3,B_Ref. Legislativa,38
4,I_Doutrinador,62
5,I_Pessoa,59
6,I_Precedente,218
7,I_Ref. Legislativa,242
8,O,5196
